# Deep Learning RNN Model for Used Motorcycle Price Prediction
## IS794 AL - Deep Learning (Lab)
Kelompok B: <br>
- Daffa Kaisha Pratama Chandra (00000055770)
- Justine Donovan (00000063202)
- Marsella Patricia (00000063226)
- Reina Ratu Balqis (00000065099)
---
### Import Libraries

In [1]:
# Basic library
import pandas as pd
import numpy as np
from scipy import stats

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Deep learning
import tensorflow as tf
from tensorflow import keras

# Ignore warning
import warnings
warnings.simplefilter("ignore")

---
Baca Dataset

In [7]:
motor = pd.read_csv("used_motorcycle.csv", index_col=False)
motor

,nama_motor,lokasi,harga,deskripsi,kapasitas_mesin,tahun,transmisi,kilometer,url,merk
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
5,Yamaha Lexi S,Kota Medan,"Rp17,800,000","Ss lengkap , pajak plat hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
6,Yamaha Jupiter Mx Cw,Kota Medan,"Rp12,500,000","Ss lengkap , nego di tempat",125 CC,2017.0,Manual,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
7,Yamaha All New Nmax 155,Kota Medan,"Rp27,300,000","Ss lengkap , plat pajak hidup , nego di tempat",155 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
8,Yamaha Nmax Abs,Kota Medan,"Rp27,300,000","Ss lengkap , plat pajak hidup , nego di tempat",155 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
9,Yamaha Fino Sporty,Kota Medan,"Rp14,800,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2018.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha


In [6]:
print(f"Dataset memiliki {motor.shape[1]} kolom dan {motor.shape[0]} baris.")

Dataset memiliki 10 kolom dan 48 baris.


Penjelasan singkat mengenai dataset yang digunakan: <br>
Dataset ini merupakan hasil web scraping dari website https://momotor.id/

---
#### Exploratory Data Analysis <br>
Pada tahap EDA, akan dilakukan ....

---
#### Data Pre-processing
Pada tahap ini, akan dilakukan:
- <i>remove missing values</i>
- 

---
#### Modeling
Pada tahap ....

Tuning Hyperparameter <br>
....

Finalisasi Model
....

---
#### Evaluasi

Terlihat model .....

---
#### Save model
Model yang dengan akurasi tertinggi akan disimpan untuk di <i> deploy </i>